This notebook is to demostrate how you can read and analyze any document - PDF, HTML, PPT, Image etc. It uses python open source package 'Unstructured', followed by Open AI LLM to perform end to end retrieval. 

In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [15]:
#pip install unstructured-client

In [17]:
#pip install unstructured

In [19]:
from IPython.display import JSON

import json
import os
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements, elements_to_json

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements

import chromadb

In [21]:
from Utils import Utils
import os
utils = Utils()

DLAI_API_KEY = utils.get_dlai_api_key()
DLAI_API_URL = utils.get_dlai_url()
#os.environ['DLAI_API_KEY'] = 'EhA5zYMRXZlZeR8V46ZEsUKziVRxag'
#os.environ['DLAI_API_URL'] = 'https://api.unstructured.io/general/v0/general'

s = UnstructuredClient(
    api_key_auth=DLAI_API_KEY,
    server_url=DLAI_API_URL,
)

In [23]:
os.getcwd()

'C:\\Users\\vnirwan\\Desktop\\Vaishu\\AI Course\\GDrive'

## Example Document: Medium Blog HTML Page

In [29]:
filename = "Forgit/Ethical_Challenges_Medium.html"
elements_html = partition_html(filename=filename)

We can also get these elements in a json format

In [34]:
element_dict = [el.to_dict() for el in elements_html]
example_output = json.dumps(element_dict[11:15], indent=2)
print(example_output)

[
  {
    "type": "Title",
    "element_id": "8d23208bb22c5e662570b8a8a84db477",
    "text": "1. Privacy at Risk: The Dangers of Data Overreach \ud83d\udd0d",
    "metadata": {
      "category_depth": 1,
      "last_modified": "2024-09-11T23:34:23",
      "languages": [
        "eng"
      ],
      "parent_id": "8c55b5e80e73435e369db4ba08b0f337",
      "file_directory": "Forgit",
      "filename": "Ethical_Challenges_Medium.html",
      "filetype": "text/html"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "0318f52ce3254b0c09d157517a4d133a",
    "text": "\ud83d\udccdIssue 1: Data Collection Overload AI thrives on data, and often, this data is personal. AI systems collect vast amounts of personal information from everyday interactions \u2014 raising eyebrows about how this information is stored, used, and protected.",
    "metadata": {
      "emphasized_text_contents": [
        "Issue 1: Data Collection Overload"
      ],
      "emphasized_text_tags": [
        "b"
     

In [38]:
filename = "Forgit/Presentation.pptx"
elements_ppt = partition_pptx(filename=filename)


In [40]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
import os
import openai
import sys
import chromadb

from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
sys.path.append('../..')

In [41]:
elements = chunk_by_title(elements_html+elements_ppt)
elements = chunk_by_title(elements_ppt)

In [42]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [46]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ["OPENAI_API_KEY"] = "yourkey"

In [48]:
embeddings = OpenAIEmbeddings()

In [49]:
vectorstore = Chroma.from_documents(documents, embeddings)

INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [52]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [54]:
template = """You are an AI assistant for answering questions related to the results and conclusions we got from these analyses about SIMS. Try to understand the context of a conclusion while using the information from these files. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about SIMS, politely inform them that you are tuned to only answer questions about Donut.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [56]:
llm = OpenAI(temperature=0)

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

C:\Users\vnirwan\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\vnirwan\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_history_aware_retriever together with create_retrieval_chain (see example in docstring) instead.
  warn_deprecated(


In [58]:
qa_chain.invoke({
    "question": " What are key ethical implications of AI’s rise?",
    "chat_history": []
})["answer"]

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


" Some key ethical implications of AI's rise include potential bias and discrimination, lack of transparency and accountability, job displacement, and privacy and security concerns.\nSOURCES: Presentation.pptx"

In [60]:
qa_chain.invoke({
    "question": "How does Artifical Intelligence work",
    "chat_history": []
})["answer"]

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


' Artificial Intelligence works by using machine learning to perform calculations and make decisions. It has the potential to be biased and has been used in the field of cybersecurity to detect and prevent attacks. \nSOURCES: Presentation.pptx'